In [1]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline
%cd '../HOTS'

/Users/joe/Documents/boulot/git/homhots/HOTS/HOTS


In [2]:
import os
import pickle
import datetime
from mix_Network import *

dataset = 'nmnist'
records_path = '../Records'
timestr = datetime.datetime.now().date().isoformat()
timestr = '2020-12-21'
verbose = True

homeo = True
sigma = None
pooling = False
homeinv = False
jitter = False
tau = 5
nb_train = 500
nb_test = 100
krnlinit = 'first'
nblay = 3
nbclust = 4

ds = 75
nb_train = 7500//ds
nb_test = 2500//ds

In [3]:
def get_nmnist(NbTrainingData, NbTestingData):
    class eventV(object):
            def __init__(self, nbevt):
                self.time = np.zeros((nbevt))
                self.address = np.zeros((nbevt, 2))
                self.polarity = np.zeros((nbevt))
                self.ImageSize = [34,34]
                self.ListPolarities = [0,1]
                
    learningset = tonic.datasets.NMNIST(save_to='../Data/',
                                train=False,
                                transform=None)
    loader = tonic.datasets.DataLoader(learningset, shuffle=True)
    
    eventslist = [next(iter(loader)) for i in range(NbTrainingData)]
    nbtot = 0
    for i in range(NbTrainingData):
        nbtot+=eventslist[i][0].shape[1]
    events_train = eventV(nbtot)
    labels_train = []
    idx = 0
    for i in range(NbTrainingData):
        for j in range(eventslist[i][0].shape[1]):
            events_train.address[idx,0] = eventslist[i][0][0,j,0].item()
            events_train.address[idx,1] = eventslist[i][0][0,j,1].item()
            events_train.time[idx] = eventslist[i][0][0,j,2].item()*1e-6
            events_train.polarity[idx] = eventslist[i][0][0,j,3].item()
            labels_train.append(eventslist[i][1])
            idx+=1
            
    eventslist = [next(iter(loader)) for i in range(NbTestingData)]
    nbtot = 0
    for i in range(NbTestingData):
        nbtot+=eventslist[i][0].shape[1]
    events_test = eventV(nbtot)
    labels_test = []
    idx = 0
    for i in range(NbTestingData):
        for j in range(eventslist[i][0].shape[1]):
            events_test.address[idx,0] = eventslist[i][0][0,j,0].item()
            events_test.address[idx,1] = eventslist[i][0][0,j,1].item()
            events_test.time[idx] = eventslist[i][0][0,j,2].item()*1e-6
            events_test.polarity[idx] = eventslist[i][0][0,j,3].item()
            labels_test.append(eventslist[i][1])
            idx+=1
    events_cluster = []
    return [events_train, events_test, events_cluster, labels_train, labels_test]

#### Building matrix for logistic regression
def gather_data(events_in, labels_in,
                tau_cla=150, # characteristic time of a digit
                sample_events=50, sample_space = 1,
                verbose=False, debug=False):

    tau_cla *= 1e3 # to enter tau in ms
    n_events = events_in.time.shape[0]

    c_int = lambda n, d : ((n - 1) // d) + 1
    data = np.zeros((c_int(events_in.ImageSize[0], sample_space),
                     c_int(events_in.ImageSize[1], sample_space),
                     len(events_in.ListPolarities))) #tmp data

    X = np.zeros((c_int(n_events, sample_events), len(data.ravel())))
    y = np.zeros((c_int(n_events, sample_events), ))

    for i_event in range(1, n_events):
        if events_in.time[i_event]<events_in.time[i_event-1]:
            data = np.zeros((c_int(events_in.ImageSize[0], sample_space),
                     c_int(events_in.ImageSize[1], sample_space),
                     len(events_in.ListPolarities))) #tmp data 

        data *= np.exp(-(events_in.time[i_event]-events_in.time[i_event-1])/tau_cla)
        
        x_pos = events_in.address[i_event, 0]//sample_space
        y_pos = events_in.address[i_event, 1]//sample_space
        p = events_in.polarity[i_event]
        data[int(x_pos), int(y_pos), int(p)] = 1.

        if i_event % sample_events == sample_events//2 :
            if debug:
                print(f'DEBUG {i_event} {i_event//sample_events} ')
                print(f'DEBUG {y[i_event//sample_events]}   ')
                print(f'DEBUG  {labels_in[i_event]} ')
            X[i_event//sample_events, :] = data.ravel()
            y[i_event//sample_events] = labels_in[i_event]
            
    if verbose: print('Number of events: ' + str(X.shape[0])+' - Number of features: ' + str(X.shape[1]))

    return X, y

def get_events(tau, krnlinit, nblay, nbclust, homeo, records_path, timestr, dataset, sigma, homeinv, jitter, nb_train, nb_test):
    hotshom = network(krnlinit=krnlinit, tau=tau, nblay=nblay, nbclust=nbclust, homeo=homeo, sigma=sigma, homeinv=homeinv, jitter=jitter)
    arch = [hotshom.L[i].kernel.shape[1] for i in range(len(hotshom.L))]
    fname = f'{records_path}/EXP_03_NMNIST/{timestr}_hots_{dataset}_{arch}_{tau}_{homeo}_{sigma}_{homeinv}_{jitter}'
    print(fname)
    if not os.path.isfile(fname+'_model.pkl'):
        loaderhom, order = hotshom.learning1by1(dataset=dataset)
        with open(fname+'_model.pkl', 'wb') as file:
            pickle.dump([hotshom, loaderhom, order], file, pickle.HIGHEST_PROTOCOL)
    else:
        with open(fname+'_model.pkl', 'rb') as file:
            hotshom, loaderhom, order = pickle.load(file)
            
    if not os.path.isfile(fname+f'_evout_{nb_train}_{nb_test}.pkl'):       
        _, loaderhom, out_train = hotshom.training(loaderhom, order, nb_digit = nb_train, LR=True)
        _, loaderhom, out_test = hotshom.training(loaderhom, order, nb_digit = nb_test, LR=True)
        with open(fname+f'_evout_{nb_train}_{nb_test}.pkl', 'wb') as file:
            pickle.dump([out_train, out_test], file, pickle.HIGHEST_PROTOCOL)
    else:
        with open(fname+f'_evout_{nb_train}_{nb_test}.pkl', 'rb') as file:
            out_train, out_test = pickle.load(file)
    return out_train[0], out_test[0], out_train[1], out_test[1]


# ### Performingxlogistic regression
#
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html
#

# from sklearn.linear_model import LogisticRegression as LR
from sklearn.linear_model import LogisticRegressionCV as LR
# 
opts_LR = dict(max_iter=2000, # random_state=0,
               n_jobs=-1, class_weight='balanced')
#opts_LR['Cs'] = 5
opts_LR['Cs'] = 32
# TODO for a publication use 100 from 10^{-10} to 10

from sklearn.model_selection import train_test_split

def tic():
    global ttic
    ttic = time.time()
def toc():
    print(f'Done in {time.time() - ttic:.3f} s')


### Performing logistic regression on raw input spikes

In [11]:
tic()
dataset = get_nmnist(nb_train, nb_test)
events_train, events_test, events_cluster, labels_train, labels_test = dataset
toc()

Using downloaded and verified file: ../Data/nmnist_test.zip
Extracting ../Data/nmnist_test.zip to ../Data/
Done in 35.425 s


In [12]:
tic()
X_train, y_train = gather_data(events_train, labels_train, verbose=verbose)
toc()

Number of events: 8547 - Number of features: 2312
Done in 6.284 s


In [13]:
tic()
lr = LR(**opts_LR).fit(X_train, y_train)
print(f'Classification score for raw input is {lr.score(X_train, y_train):.3f} (train)')
toc()

Classification score for raw input is 0.999 (train)
Done in 650.001 s


In [14]:
tic()
X_test, y_test = gather_data(events_test, labels_test, verbose=verbose)
print(f'Classification score for raw input is {lr.score(X_test, y_test):.3f} (test)')
toc()

Number of events: 2790 - Number of features: 2312
Classification score for raw input is 0.543 (test)
Done in 1.601 s


### Performing logistic regression with or without homeostasis

In [4]:
for homeo in [False, True]:
    print(40*'-')
    print(f'homeo={homeo}')
    print(40*'-')
    events_train_o, events_test_o, labels_train, labels_test = get_events(tau, krnlinit, nblay, nbclust, homeo, records_path, timestr, dataset, sigma, homeinv, jitter, nb_train, nb_test)
        
    X_train, y_train = gather_data(events_train_o, labels_train, verbose=verbose)

    lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
    print(f'Classification score for homeo = {homeo} is {lr.score(X_train, y_train):.3f} (train)')
    
    X_test, y_test = gather_data(events_test_o, labels_test, verbose=verbose)
    print(f'Classification score for homeo = {homeo} is {lr.score(X_test, y_test):.3f} (test)')

----------------------------------------
homeo=False
----------------------------------------
../Records/EXP_03_NMNIST/2020-12-21_hots_nmnist_[4, 8, 16]_5_False_None_False_False



Number of events: 7853 - Number of features: 18496


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 48.7min finished


Classification score for homeo = False is 1.000 (train)
Number of events: 2615 - Number of features: 18496
Classification score for homeo = False is 0.556 (test)
----------------------------------------
homeo=True
----------------------------------------
../Records/EXP_03_NMNIST/2020-12-21_hots_nmnist_[4, 8, 16]_5_True_None_False_False



Number of events: 8087 - Number of features: 18496


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 42.9min finished


Classification score for homeo = True is 0.999 (train)
Number of events: 2407 - Number of features: 18496
Classification score for homeo = True is 0.742 (test)
